In [16]:
import mysql.connector
import os
import random
from faker import Faker
import re

In [17]:
fake = Faker()

class ATSDatabase:
    def __init__(self):
        self.conn = mysql.connector.connect(
            host='ats-database-fareltaza35-4318.b.aivencloud.com',
            user='avnadmin',
            password='AVNS_wk8smk5E9W1DPhrWb3D',
            database='defaultdb',
            port=24585,
        )
        self.cursor = self.conn.cursor()
        self.create_tables()
    
    def create_tables(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS applicantprofile (
                applicant_id INT AUTO_INCREMENT PRIMARY KEY,
                first_name VARCHAR(50),
                last_name VARCHAR(50),
                date_of_birth DATE,
                address VARCHAR(255),
                phone_number VARCHAR(20)
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS applicationdetail (
                detail_id INT AUTO_INCREMENT PRIMARY KEY,
                applicant_id INT NOT NULL,
                applicant_role VARCHAR(100),
                cv_path TEXT,
                FOREIGN KEY (applicant_id) REFERENCES applicantprofile(applicant_id)
            )
        ''')
        self.conn.commit()

    def add_applicant(self, first_name, last_name, date_of_birth, address, phone_number):
        self.cursor.execute('''
            INSERT INTO applicantprofile (first_name, last_name, date_of_birth, address, phone_number)
            VALUES (%s, %s, %s, %s, %s)
        ''', (first_name, last_name, date_of_birth, address, phone_number))
        self.conn.commit()

    def get_applicant_id(self, first_name, last_name):
        self.cursor.execute('''
            SELECT applicant_id FROM applicantprofile
            WHERE first_name=%s AND last_name=%s
        ''', (first_name, last_name))
        result = self.cursor.fetchone()
        return result[0] if result else None

    def get_or_create_applicant(self, first_name, last_name, date_of_birth="", address="", phone_number=""):
        applicant_id = self.get_applicant_id(first_name, last_name)
        if not applicant_id:
            self.add_applicant(first_name, last_name, date_of_birth, address, phone_number)
            applicant_id = self.cursor.lastrowid
        return applicant_id

    def add_application(self, applicant_id, applicant_role, cv_path):
        self.cursor.execute('''
            INSERT INTO applicationdetail (applicant_id, applicant_role, cv_path)
            VALUES (%s, %s, %s)
        ''', (applicant_id, applicant_role, cv_path))
        self.conn.commit()

    def save_cv_to_database(self, first_name, last_name, applicant_role, cv_path):
        date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=60).strftime("%Y-%m-%d")
        address = fake.address().replace('\n', ', ')
        phone_number = re.sub(r'[^\d+]', '', fake.phone_number())[:20]
        applicant_id = self.get_or_create_applicant(first_name, last_name, date_of_birth, address, phone_number)
        self.add_application(applicant_id, applicant_role, cv_path)

    def get_all_applicants(self):
        self.cursor.execute('SELECT * FROM applicantprofile')
        return self.cursor.fetchall()

    def get_all_applications(self):
        self.cursor.execute('SELECT * FROM applicationdetail')
        return self.cursor.fetchall()

    def update_applicant(self, applicant_id, first_name, last_name, date_of_birth, address, phone_number):
        self.cursor.execute('''
            UPDATE applicantprofile
            SET first_name=%s, last_name=%s, date_of_birth=%s, address=%s, phone_number=%s
            WHERE applicant_id=%s
        ''', (first_name, last_name, date_of_birth, address, phone_number, applicant_id))
        self.conn.commit()

    def delete_applicant(self, applicant_id):
        self.cursor.execute('DELETE FROM applicantprofile WHERE applicant_id=%s', (applicant_id,))
        self.conn.commit()

    def update_application(self, detail_id, applicant_id, applicant_role, cv_path):
        self.cursor.execute('''
            UPDATE applicationdetail
            SET applicant_id=%s, applicant_role=%s, cv_path=%s
            WHERE detail_id=%s
        ''', (applicant_id, applicant_role, cv_path, detail_id))
        self.conn.commit()

    def delete_application(self, detail_id):
        self.cursor.execute('DELETE FROM applicationdetail WHERE detail_id=%s', (detail_id,))
        self.conn.commit()

    def close(self):
        self.conn.close()

In [18]:
def read_file(root_dir):
    ats_db = ATSDatabase()  

    first_names = ['John', 'Supri', 'Jane', 'Alex', 'Emily', 'Asep', 'Jajang', 'Chris', 'Katie', 'Dadang', 'Michael', 'Sara', 'Cecep', 'David', 'Laura']
    last_names = ['Smith', 'Saepuloh', 'Johnson', 'Williams', 'Suherman', 'Brown', 'Jones', 'Garcia', 'Sumarna', 'Miller', 'Davis', 'Santosa', 'Martinez', 'Hernandez']
    
    for category in os.listdir(root_dir):
        category_path = os.path.join(root_dir, category)

        if os.path.isdir(category_path): 
            print(f"Processing category: {category}")

            pdf_files = [f for f in os.listdir(category_path) if f.endswith('.pdf')][:20]

            for pdf_file in pdf_files:
                pdf_path = os.path.join(category_path, pdf_file)
                print(f"Processing CV: {pdf_file}")

                ats_db.save_cv_to_database(first_name=random.choice(first_names), 
                                            last_name=random.choice(last_names), 
                                            applicant_role=category, 
                                            cv_path=pdf_path)

    ats_db.close()


In [19]:
read_file('archive\data\data')

Processing category: ACCOUNTANT
Processing CV: 10554236.pdf
Processing CV: 10674770.pdf
Processing CV: 11163645.pdf
Processing CV: 11759079.pdf
Processing CV: 12065211.pdf
Processing CV: 12202337.pdf
Processing CV: 12338274.pdf
Processing CV: 12442909.pdf
Processing CV: 12780508.pdf
Processing CV: 12802330.pdf
Processing CV: 13072019.pdf
Processing CV: 13130984.pdf
Processing CV: 13294301.pdf
Processing CV: 13491889.pdf
Processing CV: 13701259.pdf
Processing CV: 14055988.pdf
Processing CV: 14126433.pdf
Processing CV: 14224370.pdf
Processing CV: 14449423.pdf
Processing CV: 14470533.pdf
Processing category: ADVOCATE
Processing CV: 10186968.pdf
Processing CV: 10344379.pdf
Processing CV: 10659182.pdf
Processing CV: 10818478.pdf
Processing CV: 11174187.pdf
Processing CV: 11188218.pdf
Processing CV: 11773767.pdf
Processing CV: 11963737.pdf
Processing CV: 12171093.pdf
Processing CV: 12544735.pdf
Processing CV: 13072354.pdf
Processing CV: 13115648.pdf
Processing CV: 13342150.pdf
Processing CV:

In [21]:
def verify_database():
    conn = mysql.connector.connect(
        host='ats-database-fareltaza35-4318.b.aivencloud.com',
        user='avnadmin',
        password='AVNS_wk8smk5E9W1DPhrWb3D',
        database='defaultdb',
        port=24585
    )
    cursor = conn.cursor()

    print("\n--- Applicants ---")
    cursor.execute('SELECT * FROM applicantprofile')
    applicants = cursor.fetchall()
    for applicant in applicants:
        print(applicant)

    print("\n--- Applications ---")
    cursor.execute('SELECT * FROM applicationdetail')
    applications = cursor.fetchall()
    for application in applications:
        print(application)

    conn.close()

verify_database()



--- Applicants ---
(1, 'David', 'Martinez', datetime.date(1971, 11, 15), '19669 Eugene Forge, Porterchester, WY 93566', '(933)894-7108x012')
(2, 'Asep', 'Suherman', datetime.date(2001, 12, 24), '9429 Tracy Harbor, Meghanside, AZ 94870', '755.499.4125x4553')
(3, 'Michael', 'Williams', datetime.date(1970, 5, 6), '96841 Adam Pine, Glennbury, VA 33711', '942-226-3363')
(4, 'Sara', 'Saepuloh', datetime.date(1967, 5, 16), '543 Michelle Keys, Port Brittneybury, NE 76973', '0017649938349')
(5, 'Jane', 'Martinez', datetime.date(1985, 5, 10), '8491 Lutz Terrace, New Mariashire, NE 09019', '575656341465838')
(6, 'Asep', 'Sumarna', datetime.date(1964, 12, 21), '17000 Rodriguez Roads, New Erinland, SD 70820', '744766708492339')
(7, 'Jajang', 'Suherman', datetime.date(2001, 10, 11), '20291 Jennifer Dale, Ryanburgh, WA 66253', '+19368274385205')
(8, 'Jajang', 'Johnson', datetime.date(2005, 6, 8), '03965 Brenda Isle, East Anthony, VI 16415', '+1259265086746885')
(9, 'Jajang', 'Davis', datetime.date(1